### 피아노 Midi 사이트
- https://www.freepianotutorials.net/2023/12/ludwig-goransson-can-you-hear-music.html#more
- http://www.piano-midi.de/bach.htm

### Ⅰ. 라이브러리

In [2]:
import os
import editdistance
import numpy as np
import pandas as pd

import mido
from mido import MidiFile, MidiTrack, MetaMessage, Message
from tqdm.notebook import tqdm

In [3]:
data_folder = "midi_data"

In [4]:
tick_base = True

In [5]:
###################################################
############  트랙 기본 정보
###################################################

def print_midi_info(mid):
    
    print("파일 이름: ", mid.filename)
    print("총 재생 시간: ", mid.length)
    
    if mid.tracks:
        total_tracks = len(mid.tracks)
        
        print("-"*70)
        print("트랙 이름: ", mid.tracks[0].name)
        print("총 트랙의 수: ", total_tracks)
        
        for i in range(total_tracks):
            if mid.tracks[i].name:
                track_name = mid.tracks[i].name
            else:
                track_name = None
            
            print(f"[{i+1}]. {mid.tracks[i].name}")
        
    else:
        print("트랙이 없습니다")

In [6]:
def load_midi_data(input_name, target_name):
    input_path  = os.path.join(data_folder, input_name)
    target_path = os.path.join(data_folder, target_name)
    
    input_mid  = mido.MidiFile(input_path)
    target_mid = mido.MidiFile(target_path)
    
    print("="*20, "[Input Midi Data]", "="*20)
    print_midi_info(input_mid)
    
    print()
    
    print("="*20, "[Target Midi Data]", "="*20)
    print_midi_info(target_mid)
    
    return input_mid, target_mid

In [7]:
input_name  = "butterfly/butterfly_input_data.mid"
target_name = "butterfly/butterfly_target_data.mid"

input_mid, target_mid = load_midi_data(input_name, target_name)

==================== [Input Midi Data] ====================
파일 이름:  midi_data\butterfly/butterfly_input_data.mid
총 재생 시간:  27.00000000000003
----------------------------------------------------------------------
트랙 이름:  butterfly
총 트랙의 수:  2
[1]. butterfly
[2]. PIANO

==================== [Target Midi Data] ====================
파일 이름:  midi_data\butterfly/butterfly_target_data.mid
총 재생 시간:  27.0
----------------------------------------------------------------------
트랙 이름:  butterfly
총 트랙의 수:  2
[1]. butterfly
[2]. PIANO


In [11]:
for msg in target_mid:
    if msg.type =='control_change':
        print(msg)

control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0


In [9]:
target_mid

MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    MetaMessage('track_name', name='butterfly', time=0),
    MetaMessage('set_tempo', tempo=500000, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('end_of_track', time=0)]),
  MidiTrack([
    MetaMessage('track_name', name='PIANO', time=0),
    Message('control_change', channel=0, control=7, value=100, time=0),
    Message('control_change', channel=0, control=10, value=64, time=0),
    Message('note_on', channel=0, note=69, velocity=100, time=960),
    Message('note_off', channel=0, note=69, velocity=64, time=240),
    Message('note_on', channel=0, note=70, velocity=100, time=0),
    Message('note_off', channel=0, note=70, velocity=64, time=240),
    Message('note_on', channel=0, note=41, velocity=86, time=0),
    Message('note_on', channel=0, note=72, velocity=87, time=0),
    Message('note_on', channel=0, note=48, velocity=91

### Ⅱ. 데이터 추출하기

In [358]:
def load_mid_Info_from_csv(input_mid, target_mid):
    
    input_name  = input_mid.filename
    target_name = target_mid.filename
    
    input_csv  = input_name.split(".")[0]+".csv"
    target_csv = target_name.split(".")[0]+".csv"
    
    input_info = pd.read_csv(input_csv)
    target_info = pd.read_csv(target_csv)
    
    #input_info = pd.read_csv(input_csv, index_col = 0)
    #target_info = pd.read_csv(target_csv, index_col = 0)
    
    return input_info, target_info


#### [1]. PreCheck
- track 길이
- track 이름
- 음의 길이

In [361]:
def pre_check(input_mid, target_mid):
    input_track_len  = len(input_mid.tracks)
    target_track_len = len(target_mid.tracks)
    
    # track 길이 비교
    if input_track_len != target_track_len:
        print(f"input track len({input_track_len}) != target track len({target_track_len})")
        return False
    
    # track_name 비교
    input_track_name  = [ msg.name for msg in input_mid if msg.type == 'track_name']
    target_track_name = [ msg.name for msg in target_mid if msg.type == 'track_name']
    
    ## if input_track_name != target_track_name:
    if input_track_name == target_track_name:
        print(f"input track name({input_track_name}) != target track name({target_track_name})")
        return False
    
    # track 시간 비교 ( time_diff_interval 이상 비교 불가)
    input_mid_time  = input_mid.length
    target_mid_time = target_mid.length
    
    time_diff_interval = 20
    
    if abs(input_mid_time - target_mid_time) > time_diff_interval:
        print(f"input time({input_mid_time}) != target time({target_mid_time})")
        return False
    
    return True


#### [2]. MetaMessage

#### [3]. Message
- mean_velocity ( 평균 세기 ) : (( 셈여림 ))에 활용
- diff_velocity ( 세기 변화 ) 
    - 현재 노트의 평균 세기 - 이전 노트의 평균 세기
    - 음표의 세기, 빠르기의 변화에 활용
- dynamic ( 셈여림 )
|셈여림(dynamic)|정의|평균 velocity 범위|
|:--:|:--:|:--:|
|ppp(피아니시시모)|아주 여리게|0~35|
|pp(피아니시모)|매우 여리게|36~48|
|p(피아노)|여리게|49~61|
|mp(메조 피아노)|조금 여리게|62~74|
|mf(메조 포르테)|조금 세게|75~87|
|f(포르테)|세게|88~100|
|ff(포르티시모)|매우 세게|101~113|
|fff(포르티시시모)|아주 세게|114~127|

In [211]:
def create_df_info():
    df_col = ['sec', 'tick', 'bpm', 
              'msg_type', 'channel', 'note', 'velocity', 'dynamic', 'accent', 'count',
              'main_vol','depth', 'pedal', 'pan', 'tempo']
    df = pd.DataFrame(columns = df_col)
    
    df_tempo_col = ['tempo', 'tick']
    df_tempo = pd.DataFrame(columns = df_tempo_col)

    df_info = { MetaMessage:{}, Message:df }
    
    return df_info, df_tempo

def info_to_list(cur_info):
    
    cur_temp = [cur_info['sec'], cur_info['tick'], cur_info['bpm'],
                cur_info['msg_type'], cur_info['channel'], cur_info['note'], 
                cur_info['velocity'], cur_info['dynamic'], cur_info['accent'], cur_info['count'],
                cur_info['main_vol'], cur_info['depth'], cur_info['pedal'], cur_info['pan'], cur_info['tempo']]
    
    return cur_temp
                

def initialize_cur_info():
    cur_info = {'sec':0, 'tick':0, 'bpm':0,
                'msg_type': [], 'channel':[], 'note':[],  'velocity':[], 'dynamic':"", 'accent':0, 'count':0,
                'main_vol':-1, 'depth':-1, 'pedal':-1, 'pan':-1, 'tempo':0 }
    return cur_info

def check_not_list(msg_list):
    if type(msg_list) == list:
        return msg_list
    else:
        msg_list = [msg_list]
        return msg_list

In [136]:
def tempo_info_list(mid_info, tempo_info):
    
    #default tempo setting
    if 'set_tempo' not in mid_info[MetaMessage]:
        msg = mido.MetaMessage('set_tempo', tempo = 500000)
        
        mid_info[MetaMessage][msg.type] = [msg]
    
    tempo_list = mid_info[MetaMessage]['set_tempo']
    
    tempo_tick = 0
    
    for idx, msg in enumerate(tempo_list):
        tempo_tick += msg.time
            
        if idx==0:
            tempo_info.loc[idx,'tempo'] = msg.tempo
        else:
            tempo_info.loc[idx, 'tempo'] = msg.tempo
            tempo_info.loc[idx-1, 'tick'] = tempo_tick
            
    tempo_info.loc[tempo_info.shape[0] - 1, 'tick'] = -1
                    
    return tempo_info

def find_tempo(tick, tempo_info):
    tempo_idx = 0
    last_tempo_idx = tempo_info.shape[0]
    
    cur_tempo = -1
    
    for tempo_idx in range(last_tempo_idx):
        if tempo_info.loc[tempo_idx, 'tick'] > tick:
            cur_tempo = tempo_info.loc[tempo_idx, 'tempo']
            break
            
    if cur_tempo == -1:
        cur_tempo = tempo_info.loc[last_tempo_idx - 1, 'tempo']
        
    return cur_tempo

In [224]:
###################################################
############  셈여림 정보
###################################################
def distribute_dynamic(mean_velocity):
    dynamic = ''
    if 0 <= mean_velocity < 36:
        dynamic = 'ppp'
    elif 36 <= mean_velocity < 49:
        dynamic = 'pp'
    elif 49 <= mean_velocity < 62:
        dynamic = 'p'
    elif 62 <= mean_velocity < 75:
        dynamic = 'mp'
    elif 75 <= mean_velocity < 88:
        dynamic = 'mf'
    elif 88 <= mean_velocity < 101:
        dynamic = 'f'
    elif 101 <= mean_velocity < 114:
        dynamic = 'ff'
    elif 114 <= mean_velocity <= 127:
        dynamic = 'fff'
    
    return dynamic

###################################################
############  셈여림, 악센트 계산 함수
###################################################
def calculate_dynamic_accent(cur_info, msg_df):
    
    last_idx = msg_df.shape[0]
    
    if last_idx>0:
        prev_velo = msg_df.loc[last_idx-1, 'velocity']
        prev_avg_velo = sum(prev_velo) / len(prev_velo) if prev_velo else 0
    else:
        prev_avg_velo = 0
        
    if cur_info['count'] > 0 :
        mean_velocity = sum(cur_info['velocity']) / len(cur_info['velocity'])
        diff_velocity = mean_velocity - prev_avg_velo
    else:
        mean_velocity = 0
        diff_velocity = 0
        
    dynamic = distribute_dynamic(mean_velocity)
    accent = 1 if 76 <= mean_velocity <= 127 and mean_velocity > 1.2 * prev_avg_velo else 0
    
    return dynamic, accent
    

In [265]:
def control_type(msg):
    
    ctl_type = None
    
    if msg.control == 1:
        ctl_type = 'modulation'
        
    elif msg.control == 7:
        ctl_type = 'main_vol'
        
    elif msg.control == 10:
        ctl_type = 'pan'
        
    elif msg.control == 64:
        ctl_type = 'pedal'
        
    elif msg.control >= 91 and msg.control <= 93:
        ctl_type = 'depth'
        
    return ctl_type

In [262]:
def process_msg(msg, info):
    
    if msg.type == 'note_on' or msg.type == 'note_off':
        
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_type = 'note_off'
        else:
            msg_type = msg.type
                    
        info['msg_type'].append(msg_type)
        info['channel'].append(msg.channel)
        info['note'].append(msg.note)
        info['velocity'].append(msg.velocity)
                    
        info['count']+=1
        
    elif msg.type == 'program_change':
        print("program_change")
        
    elif msg.type == 'control_change':
        
        ctl_type = control_type(msg)
        
        # 수집하지 않은 control_type
        if ctl_type == None:
            return info
        
        if info[ctl_type] == -1:
            info[ctl_type] = msg.value
        else:
            info[ctl_type] = check_not_list(info[ctl_type])
            info[ctl_type].append(msg.value)
    
    return info

In [359]:
def extract_mid_data(mid):
    
    mid_info, tempo_info = create_df_info()
    tick_base = True
    
    for idx, track in enumerate(mid.tracks):
        
        cur_tick = 0
        cur_sec  = 0
        cur_info = initialize_cur_info()
        
        for i, msg in tqdm(enumerate(track)):
            
            # MetaMessage
            if isinstance(msg, MetaMessage):
                                
                if msg.type not in mid_info[MetaMessage]:
                    mid_info[MetaMessage][msg.type] = []
                    
                mid_info[MetaMessage][msg.type].append(msg)
            
            elif isinstance(msg, Message):
                
                if msg.time > 0:
                    
                    dynamic, accent = calculate_dynamic_accent(cur_info, mid_info[Message])
                    
                    cur_info['dynamic'] = dynamic
                    cur_info['accent'] = accent
                    
                    cur_info['tempo'] = find_tempo(cur_tick, tempo_info)
                    cur_info['bpm'] = mido.tempo2bpm(cur_info['tempo'])
                    
                    cur_info['tick'] = cur_tick
                    cur_info['sec'] = mido.tick2second( cur_info['tick'], mid.ticks_per_beat, cur_info['tempo'] )
                    cur_info['sec'] = np.trunc(cur_info['sec']*1000)/1000 # 소수 셋째자리까지 버림
                    
                    cur_temp = info_to_list(cur_info)
                    
                    cur_tick = cur_tick + msg.time
                    last_tick = mid_info[Message].shape[0]
                    
                    if tick_base == True:
                        
                        for tick in range(last_tick, cur_tick):
                            cur_temp[0] = mido.tick2second(tick, mid.ticks_per_beat, cur_info['tempo'] ) # second
                            cur_temp[0] = np.trunc(cur_temp[0]*1000) / 1000  # 소수 셋째자리까지 버림
                            
                            cur_temp[1] = tick # tick
                            
                            mid_info[Message].loc[tick] = cur_temp
                                                    
                    else:
                        mid_info[Message].loc[last_tick] = cur_temp
                        
                    cur_info = initialize_cur_info()
                        
               
                cur_info = process_msg(msg, cur_info)
            
        if idx == 1:

            # 잔여 cur_info 추가
            dynamic, accent = calculate_dynamic_accent(cur_info, mid_info[Message])

            cur_info['dynamic'] = dynamic
            cur_info['accent'] = accent

            cur_info['tempo'] = find_tempo(cur_tick, tempo_info)
            cur_info['bpm'] = mido.tempo2bpm(cur_info['tempo'])

            cur_info['tick'] = cur_tick
            cur_info['sec'] = mido.tick2second( cur_info['tick'], mid.ticks_per_beat, cur_info['tempo'] )
            cur_info['sec'] = np.trunc(cur_info['sec']*1000)/1000 # 소수 셋째자리까지 버림

            cur_temp = info_to_list(cur_info)

            cur_tick = cur_tick + msg.time
            last_tick = mid_info[Message].shape[0]

            if tick_base == True:

                for tick in range(last_tick, cur_tick + 1):

                    cur_temp[0] = mido.tick2second(tick, mid.ticks_per_beat, cur_info['tempo'] ) # second
                    cur_temp[0] = np.trunc(cur_temp[0]*1000) / 1000  # 소수 셋째자리까지 버림

                    cur_temp[1] = tick # tick

                    mid_info[Message].loc[tick] = cur_temp

            else:
                mid_info[Message].loc[last_tick] = cur_temp

            cur_info = initialize_cur_info()
            
                
            
            
        # tempo setting
        if idx == 0:
            tempo_info_list(mid_info, tempo_info)
        if idx == 2:
            break
            
    mid_info[Message].replace(-1, 0, inplace = True)
    
    csv_name = mid.filename.split(".")[0]+".csv"
    mid_info[Message].to_csv(csv_name, index = False)
    
    return mid_info


In [345]:
def extrack_mid_info(input_mid, target_mid):
    
    input_info  = extract_mid_data(input_mid)
    target_info = extract_mid_data(target_mid)
    
    print("="*20, "  [Input_Info[Message]]  ", "="*20)
    display(input_info[Message])
    
    print()
    
    print("="*20, "  [target_Info[Message]]  ", "="*20)
    display(target_info[Message])
    
    return input_info, target_info

### Ⅲ. 실행

#### [1]. 실행 코드

In [376]:
input_name = "river_flows_in_you_test_v2.mid"
target_name = "river_flows_in_you_origin.mid"

input_mid, target_mid = load_midi_data(input_name, target_name)

==================== [Input Midi Data] ====================
파일 이름:  midi_data\river_flows_in_you_test_v2.mid
총 재생 시간:  174.7791666666652
----------------------------------------------------------------------
트랙 이름:  test_rivers_flow
총 트랙의 수:  2
[1]. test_rivers_flow
[2]. Piano

==================== [Target Midi Data] ====================
파일 이름:  midi_data\river_flows_in_you_origin.mid
총 재생 시간:  164.77394462499913
----------------------------------------------------------------------
트랙 이름:  
총 트랙의 수:  2
[1]. 
[2]. Piano


In [350]:
if pre_check(input_mid, target_mid):
    input_info, target_info = extrack_mid_info(input_mid, target_mid)
else:
    print("===== pre_check failed =====")

====================   [Input_Info[Message]]   ====================


,sec,tick,bpm,msg_type,channel,note,velocity,dynamic,accent,count,main_vol,depth,pedal,pan,tempo
0,0.000,0,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
1,0.002,1,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
2,0.004,2,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
3,0.006,3,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
4,0.008,4,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83890,174.770,83890,60.0,[],[],[],[],ppp,0,0,0,0,"[1, 1]",0,1000000
83891,174.772,83891,60.0,[],[],[],[],ppp,0,0,0,0,"[1, 1]",0,1000000
83892,174.775,83892,60.0,[],[],[],[],ppp,0,0,0,0,"[1, 1]",0,1000000
83893,174.777,83893,60.0,[],[],[],[],ppp,0,0,0,0,"[1, 1]",0,1000000



====================   [target_Info[Message]]   ====================


,sec,tick,bpm,msg_type,channel,note,velocity,dynamic,accent,count,main_vol,depth,pedal,pan,tempo
0,0.000,0,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
1,0.002,1,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
2,0.004,2,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
3,0.006,3,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
4,0.008,4,60.0,[],[],[],[],ppp,0,0,0,0,0,0,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83890,174.770,83890,60.0,[],[],[],[],ppp,0,0,0,0,"[1, 1]",0,1000000
83891,174.772,83891,60.0,[],[],[],[],ppp,0,0,0,0,"[1, 1]",0,1000000
83892,174.775,83892,60.0,[],[],[],[],ppp,0,0,0,0,"[1, 1]",0,1000000
83893,174.777,83893,60.0,[],[],[],[],ppp,0,0,0,0,"[1, 1]",0,1000000


In [364]:
if pre_check(input_mid, target_mid):
    print("===== pre_check success =====")
    input_info, target_info = load_mid_Info_from_csv(input_mid, target_mid)
else:
    print("===== pre_check failed =====")

#### [2]. 임시방편

In [399]:
input_msg_info  = pd.read_csv("midi_data/river_flows_in_you_test_v3.csv")
target_msg_info = pd.read_csv("midi_data/river_flows_in_you_origin.csv")

In [398]:
display(input_msg_info)
print("="*25)
display(target_msg_info)

,sec,tick,bpm,msg_type,channel,note,velocity,dynamic,accent,count,main_vol,depth,pedal,pan,tempo
0,0.527,253,60,['note_on'],[0],[80],[36],pp,0,1,0,0,0,0,1000000
1,0.529,254,60,['note_on'],[0],[80],[36],pp,0,1,0,0,0,0,1000000
2,0.531,255,60,['note_on'],[0],[80],[36],pp,0,1,0,0,0,0,1000000
3,0.533,256,60,['note_on'],[0],[80],[36],pp,0,1,0,0,0,0,1000000
4,0.535,257,60,['note_on'],[0],[80],[36],pp,0,1,0,0,0,0,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67507,174.637,83826,60,"['note_off', 'note_off']","[0, 0]","[69, 69]","[0, 0]",ppp,0,2,0,0,0,0,1000000
67508,174.639,83827,60,"['note_off', 'note_off']","[0, 0]","[69, 69]","[0, 0]",ppp,0,2,0,0,0,0,1000000
67509,174.641,83828,60,"['note_off', 'note_off']","[0, 0]","[69, 69]","[0, 0]",ppp,0,2,0,0,0,0,1000000
67510,174.643,83829,60,"['note_off', 'note_off']","[0, 0]","[69, 69]","[0, 0]",ppp,0,2,0,0,0,0,1000000


,sec,tick,bpm,msg_type,channel,note,velocity,dynamic,accent,count,main_vol,depth,pedal,pan,tempo
0,0.000,0,65.000135,"['note_on', 'note_on']","[0, 0]","[81, 54]","[70, 72]",mp,0,2,0,0,0,0,923075
1,0.003,1,65.000135,"['note_on', 'note_on']","[0, 0]","[81, 54]","[70, 72]",mp,0,2,0,0,0,0,923075
2,0.007,2,65.000135,"['note_on', 'note_on']","[0, 0]","[81, 54]","[70, 72]",mp,0,2,0,0,0,0,923075
3,0.010,3,65.000135,"['note_on', 'note_on']","[0, 0]","[81, 54]","[70, 72]",mp,0,2,0,0,0,0,923075
4,0.014,4,65.000135,"['note_on', 'note_on']","[0, 0]","[81, 54]","[70, 72]",mp,0,2,0,0,0,0,923075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44797,161.526,44797,65.000276,['note_off'],[0],[54],[0],ppp,0,1,0,0,0,0,923073
44798,161.530,44798,65.000276,['note_off'],[0],[54],[0],ppp,0,1,0,0,0,0,923073
44799,161.534,44799,65.000276,['note_off'],[0],[54],[0],ppp,0,1,0,0,0,0,923073
44800,242.306,44800,43.333564,['note_off'],[0],[69],[0],ppp,0,1,0,0,0,0,1384608


### Ⅳ. 비교 알고리즘

#### [0]. 사전 작업
- 정답 데이터의 tick 행 길이에 맞추는 작업
- 입력 데이터의 tick 행 길이가 적을 경우, 0 으로 처리

In [396]:
def align_and_save_dataframes(input_mid, target_mid):
    
    input_filename  = input_mid.filename
    target_filename = target_mid.filename
    
    input_file  = input_filename.split(".")[0] +".csv"
    target_file = target_filename.split(".")[0]+".csv"
    
    df_input  = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)
    
    print("Input shape: ", df_input.shape)
    print("Target shape:", df_target.shape)

    # target_file의 행 길이로 맞추기
    min_length = min(len(df_input), len(df_target))
    df_input_aligned = df_input.head(min_length)

    # target_file보다 행이 작으면 나머지 행은 0으로 채우기
    if len(df_input) < len(df_target):
        df_input_aligned = pd.concat([df_input_aligned, pd.DataFrame(0, index=range(len(df_target) - len(df_input)), columns=df_input.columns)])

    # 모든 컬럼에 대해 0으로 채우기
    for col in df_input.columns:
        if col not in df_target.columns:
            df_input_aligned[col] = 0

    # msg_type, Notes, Velocities 컬럼에 대해 빈 리스트 []를 NaN으로 처리
    for col in ['msg_type', 'channel', 'note', 'velocity']:
        df_input_aligned[col] = df_input_aligned[col].apply(lambda x: [] if x == 0 else x)

    df_input_aligned.to_csv(input_file, index=False)
    df_target.to_csv(target_file, index=False)
    
    return df_input_aligned, df_target

In [ ]:
input_msg_info, target_msg_info = align_and_save_dataframes(input_mid, target_mid)

#### [1]. 노트 정확도 ( 음정 정확도 : pitch accuracy )
- 입력 데이터와 정답 데이터의 Notes 를 비교하여, 현재 노트가 일치하면 +2 점을 부여
- [의문점]
    - 실수로 잘못 쳐서 한번 밀려서 연주한 경우 / 계속 밀리는 문제 ( -1, +1 을 해야하는 가 )

In [422]:
def calculate_note_accuracy(df_input, df_target):

    total_accuracy = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'note'])
        notes_target = eval(df_target.at[index, 'note'])

        # print('-'*20)
        # print(f'입력 데이터: {notes_input}')
        # print(f'정답 데이터: {notes_target}')
        # print('-'*20)
        
        # 0으로 된 값 리스트로 처리
        if isinstance(notes_input, int):
            notes_input = [notes_input]
        if isinstance(notes_target, int):
            notes_target = [notes_target]
        
        # 현재 행의 정확도 초기화
        row_accuracy = 0

        # 한 행에 있는 리스트들 간의 비교
        for i in range(len(notes_target)):
            for j in range(len(notes_input)):
                if notes_input[j] == notes_target[i]:
                    row_accuracy += 1
                    # print('-'*10)
                    # print(f'!!!{row_accuracy}점 추가!!!')
                    
                # 비교하는 대상들 출력
                # print(f'비교 데이터 - 입력 데이터: {notes_input[j]} / 정답 데이터: {notes_target[i]}')

        # 한 행의 정확도를 전체 정확도에 추가
        total_accuracy += row_accuracy
        # print(f'현재 점수: {total_accuracy}점')
        # print()

    # 전체 정확도 계산
    max_possible_accuracy = len(df_input)
    accuracy_percentage = min(100, (total_accuracy / max_possible_accuracy) * 100)
    
    accuracy_percentage_round = np.round(accuracy_percentage)*10000 / 10000

    return f'{accuracy_percentage_round}%'


In [424]:
accuracy = calculate_note_accuracy(input_msg_info, target_msg_info)
print(f'음정 정확도: {accuracy}')

음정 정확도: 30.0%


#### [2]. 셈여림
- 코드 작업이 더 필요한 것처럼 보임

In [425]:
def calculate_dynamic_level(input_df, target_df):

    input_counts  = input_df['dynamic'].value_counts().to_dict()
    target_counts = target_df['dynamic'].value_counts().to_dict()

    return input_counts, target_counts



In [426]:
# 셈여림
dynamic_note_score1 = calculate_dynamic_level(input_msg_info, target_msg_info)
print(dynamic_note_score1)

({'ppp': 27909, 'p': 7970, 'pp': 5529, 'mp': 3386, 'mf': 8}, {'pp': 29536, 'ppp': 14050, 'p': 1088, 'mp': 128})


In [ ]:
print(len(input_mid.tracks[1]))
print("======================")
print(len(target_mid.tracks[1]))


print(input_mid.ticks_per_beat)
print("======================")
print(target_mid.ticks_per_beat)


input_time = 0
for msg in input_mid.tracks[1]:
    if isinstance(msg, Message):
        if msg.type == "note_on" or msg.type == "note_off":
            input_time+=msg.time
        
print(input_time)

target_time = 0
for msg in target_mid.tracks[1]:
    if isinstance(msg, Message):
        if msg.type == "note_on" or msg.type == "note_off":
            target_time+=msg.time
        
print(target_time)

In [ ]:
display(input_info)
print("================")
display(target_info)